#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import load_data
from src.llm_client import LLMClient

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 指定语料库路径
data_dir = '../data/raw'

# 读取 TSV 格式语料
# 参数 limit 控制平行句对数量
# 数据存储于 DataFrame 对象
data = load_data(data_dir, limit=10)
print(f'成功读取 {len(data)} 条平行句对')

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print('数据前 5 行如下：')
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


In [3]:
# 提取指定行的数据

# 第一行
row_id = 1

# 提取原文
src_lang = 'Chinese'
src_text = data.iloc[row_id - 1]['source']

# 提取译文
tgt_lang = 'English'
tgt_text = data.iloc[row_id - 1]['target']

print(f'[ID]: {row_id:05d}')
print(f'{"-" * 80}')
print(f'[{src_lang}]：{src_text}')
print(f'[{tgt_lang}]：{tgt_text}')

[ID]: 00001
--------------------------------------------------------------------------------
[Chinese]：江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
[English]：Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.


#### **3. 加载模型**

加载大模型 API 接口

In [4]:
# 加载模型前，请登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API-Key
# 并在 config 文件中设置 LLM_API_KEY=sk-********

# 新注册用户可免费调用部分模型的 API
# 登录后可在模型服务页面查看免费模型列表

# 指定模型名称
# 可选模型包括：
# qwen-flash, qwen-plus, qwen3-max, glm-4.7, deepseek-v3.2
model = "deepseek-v3.2"

# 初始化大模型 API 接口
client = LLMClient(model)

In [5]:
# === 注意 ===
# 为节省 API 调用成本
# 将大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果

# 若需测试 API 连接是否正常
# 可修改提示词后，重新标注

#### **4. 提示词编写**

In [6]:
# 词性标注提示词

prompt = """
You are a professional corpus linguist specialized in Part-of-Speech (POS) tagging for Chinese text.

Your task is to perform POS tagging on the given text.
First tokenize the text, then assign a POS tag to each token.

text: 江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
"""

#### **4. 大模型 API 调用**

In [7]:
# 调用大模型 API
# 开始中文词性标注

result = client.get_text_response(
    prompt=prompt,
)

# 输出大模型标注结果
print(f'{"-" * 80}\n')
print(f'[大模型输出结果]\n\n{result}')

--------------------------------------------------------------------------------

[大模型输出结果]

江南/ns 近/v 海滨/n 的/u 一/m 条/q 大路/n 上/f ，/w 一/m 队/q 清兵/n 手执/v 刀枪/n ，/w 押/v 着/u 七/m 辆/q 囚车/n ，/w 冲风冒寒/v ，/w 向/p 北/f 而/c 行/v 。/w
